In [ ]:
pip install python-binance

In [ ]:
pip install TA-lib

In [ ]:
import websockets
import json
import pprint
import talib
import numpy

In [ ]:
from binance.client import Client
from binance.enums import *

In [ ]:
SOCKET = "wss://stream.binance.com:9443/ws/btcusdt@kline_1m"

API_KEY = "dV2P45YmZQ3rgQQGpFSI9aqYklLY0ke3isesjtLdy9oikJHt8KVCTCG9HRwm0i8Q"
API_SECRET = "4xTxPTdcf4Ia4blnG7TXDBAaJHsi6Qd85UoYK5C6owAFHhaqFy4WfutNjxX37cTz"
RSI_PERIOD = 14
RSI_OVERBOUGHT  = 70
RSI_OVERSOLD = 30
TRADE_SYMBOL = 'BTCUSDT'
TRADE_QUANTITY = 0.005

closes = []
in_position = False

In [ ]:
client = Client(config.API_KEY, config.API_SECRET)

In [ ]:
def order(side, quantity, symbol, order_type=ORDER_TYPE_MARKET):
    try:
        print("sending order")
        order = client.create_order(symbol=symbol, side=side, type=order_type, quantity=quantity)
        print(order)
    except Exception as e:
        print("an exception occured - {}".format(e))
        return False
    
    return True

In [ ]:

def on_open(ws):
    print('opened connection')
    
def on_close(ws):
    print('closed connection')
    
def on_message(ws, message):
    global closes, in_position
    
    print('received message')
    json_message = json.loads(message)
    pprint.pprint(json_message)
    
    candle = json_message['k']
    
    is_candle_closed = candle['x']
    close = candle['c']
    
    if is_candle_closed:
        print("candle closed at {}".format(close))
        closes.append(float(close))
        print("closes")
        print(closes)
        
    if len(closes) > RSI_PERIOD:
        np_closes = numpy.array(closes)
        rsi = talib.RSI(np_closes, RSI_PERIOD)
        print("all rsis calculated so far")
        print(rsi)
        last_rsi = rsi[-1]
        print("the current rsi is {}".format(last_rsi))
        
        
    if last_rsi > RSI_OVERBOUGHT:
        if in_position:
            print("Overbought! Sell! Sell!")
            #put binance sell logic here
            order_succeeded = order(SIDE_SELL, TRADE_QUANTITY, TRADE_SYMBOL)
            if order_succeeded:
                in_position = False
                
        else:
            print("It is overbought, we own none. Nothing to do.")
            
            
    if last_rsi < RSI_OVERSOLD:
        if in_position:
            print("It is oversold! Buy! Buy!")
            #put binance buy logic here
            order_succeeded = order(SIDE_BUY, TRADE_QUANTITY, TRADE_SYMBOL)
            if order_succeeded:
                in_position = True
                
                
ws =  websocket.WebSocketApp(SOCKET, on_open=on_open, on_close=on_close, on_message=on_message)
ws.run_forever()
